Same as final-notebook except this one doesn't keep all objects in the same matrix

In [2]:
# Reading the data

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pathlib
from copy import copy
from matplotlib import cm, colors
import cv2

emptyRatio = 47
objectwidth = 83
objectheight = 122
xspacing = 133
yspacing = 270
xstart = 293
ystart = 268
xend = 1730
yend = 1770
powderthickness = 80
endlayer = 187
objectsplit = 13
layersPerObject = endlayer // objectsplit
# Approximate one third test data
testEnd = endlayer - layersPerObject * (objectsplit // 3)

paths = pathlib.Path('./OT data 80 um/int').glob('*.tif')
paths_sorted = [x for x in paths]
paths_sorted.sort()
block = np.array([np.array(plt.imread(path)) for path in paths_sorted])
integrals = block[0:endlayer]

del paths_sorted

In [10]:
trainobjectinfo.drop(trainobjectinfo[trainobjectinfo.Label == 'KH'].index, inplace=True)
trainobjectinfo.replace('GOOD', 0, inplace=True)
trainobjectinfo.replace('LOF', 1, inplace=True)
trainobjectinfo.reset_index(drop=True, inplace=True)
testobjectinfo.drop(testobjectinfo[testobjectinfo.Label == 'KH'].index, inplace=True)
testobjectinfo.replace('GOOD', 0, inplace=True)
testobjectinfo.replace('LOF', 1, inplace=True)
testobjectinfo.reset_index(drop=True, inplace=True)

trainobjects = np.full((len(trainobjectinfo), layersPerObject, objectheight, objectwidth), np.nan)
testobjects = np.full((len(testobjectinfo), layersPerObject, objectheight, objectwidth), np.nan)

# Double check maybe that iterrows gives the correct order?
for index, object in trainobjectinfo.iterrows():
    trainobjects[index-1] = integrals[object.zstart:object.zend, object.ystart:object.yend, object.xstart:object.xend]
for index, object in testobjectinfo.iterrows():
    testobjects[index-1] = integrals[object.zstart:object.zend, object.ystart:object.yend, object.xstart:object.xend]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233


In [17]:
from sklearn import neighbors, metrics
from sklearn.model_selection import cross_val_score, LeaveOneOut
from sklearn import preprocessing
from datetime import datetime

def preprocess(object, type, sharpening):
    sharpeningKernel = np.array([   [-1, -1,  -1],
                                    [-1,  9,  -1],
                                    [ -1, -1,  -1]
    ]) if sharpening == 'diagonal' else np.array([  [0, -1,  0],
                                                    [-1, 5, -1],
                                                    [0, -1,  0]])
    sharpened = np.array([cv2.filter2D(src=image, ddepth=-1, kernel=sharpeningKernel) for image in object])
# Sharpening is done
    if type == 'scatter':
        xs = np.array(sharpened, copy=True, dtype=np.float32)
        sum = np.sum(sharpened, axis=0)
        limit = np.percentile(sum, emptyRatio)
        filter = sum >= limit
        (endLayer, _, _) = xs.shape
        filter = np.repeat([filter], endLayer, 0)
        xs[~filter] = np.nan
        return xs
    elif type == 'moran':
        xs = np.array(sharpened, copy=True, dtype=np.float32)
        sum = np.sum(xs, axis=0)
        limit = np.percentile(sum, emptyRatio)
        filter = sum >= limit
        (endLayer, _, _) = xs.shape
        filter = np.repeat([filter], endLayer, 0)
        avg = np.mean(xs, where=filter)
        stddev = np.std(xs, where=filter)
        xs = (xs - avg) / avg
        xs[~filter] = np.nan
        return xs
    elif type == 'spatstat':
        # Implement later
        print("nyi")


def calculateoutliers(object, type, neighbourhoodSetting, windowSize):
    # Step 1: calculate neighbourhood
    neighbourkernel = np.array(
        [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]]
    )/9 if neighbourhoodSetting == 'grid' else np.array(
        [[1, 2,  1],
        [2, 4, 2],
        [1, 2,  1]])/16
    flatNeighbourhood = np.array([cv2.filter2D(src=layer, ddepth=-1, kernel=neighbourkernel) for layer in object])
    (endLayer, _, _) = flatNeighbourhood.shape
    neighbourhoodValues = np.array([
        np.sum(flatNeighbourhood[layerIndex-windowSize:layerIndex], axis=0)/windowSize
        for layerIndex in range(windowSize, endLayer+1)
    ])

    # Step 2: calculate outlier
    offset = windowSize // 2
    endoffset = windowSize - offset - 1
    init = False

    if type == 'spatstat':
        print("nyi")
    else:
        outliervalues = np.full(np.shape(neighbourhoodValues), 0.0)
        # Is the axis right for moran/scatter?
        xs = object[offset:-endoffset]
        ys = neighbourhoodValues[0:-windowSize+1]
        filter = np.logical_and(np.isfinite(xs), np.isfinite(ys))
        line = np.polyfit(ys[filter].flatten(), xs[filter].flatten(), 1)
        p = np.poly1d(line)
        return xs - p(ys)

def encode(outlierobjects, type, buckets, histnormalise, minval=0, maxval=0):
    # Note: remove KF objects at some point...
    init = False

    numberOfObjects, _, _, _ = outlierobjects.shape
    X = np.full((numberOfObjects, buckets), np.nan)
    if type == 'spatstat':
        print("nyi")
    else:
        minval = np.min(outlierobjects) if minval == 0 else minval
        maxval = np.max(outlierobjects) if maxval == 0 else maxval
        for index in range(0, numberOfObjects):
            xs = outlierobjects[index]
            # filter = np.isfinite(xs)
            hist, _ = np.histogram(xs[filter], bins=buckets, range=(minval, maxval))
            X[index] = np.array(hist)
        
        if (histnormalise == 'column'):
            X = preprocessing.normalize(X, axis=0)
        elif (histnormalise == 'row'):
            X = preprocessing.normalize(X, axis=1)
    return X, minval, maxval

def classify(Xtrain, Ytrain, Xtest, Ytest, n_neighbors):
    clf = neighbors.KNeighborsClassifier(n_neighbors, weights="distance")
    clf.fit(Xtrain, Ytrain)
    yfit = clf.predict(Xtest)
    return metrics.roc_auc_score(Ytest, yfit), metrics.precision_score(Ytest, yfit), metrics.recall_score(Ytest, yfit)

In [12]:
# Tweaking parameter settings
types = ['scatter', 'moran']
# types = ['scatter', 'moran', 'spatstat']
sharpening = ['direct', 'diagonal']
neighbourhoodSetting = ['grid', 'euclidean']
windowsizes = range(1, 8, 2)
buckets = range(30, 151, 30)
histnormalise = ('none', 'row')
#bucket-lower-limit?
k = range(2, 7)

In [20]:
# Now to tie it all together...
columns = ['type', 'sharpening', 'windowSize', 'neighbourhood', 'buckets', 'histnormalise', 'k-nearest', 'auc', 'precision', 'recall']
results = pd.DataFrame(columns=columns)
Ytrain = np.array(trainobjectinfo.loc[:,"Label"])
Ytest = np.array(testobjectinfo.loc[:,"Label"])
for type in types:
    print(datetime.now(), "type: ", type)
    for sharpSetting in sharpening:
        print(datetime.now(), "sharpSetting: ", sharpSetting)
        # trainpreprocessed = np.apply_along_axis(preprocess, 1, trainobjects, type, sharpSetting)
        testpreprocessed = np.apply_along_axis(preprocess, 0, testobjects, type, sharpSetting)
        for windowsize in windowsizes:
            print(datetime.now(), "windowsize: ", windowsize)
            for nSetting in neighbourhoodSetting:
                print(datetime.now(), "nSetting: ", nSetting)
                # trainoutliers = np.apply_along_axis(calculateoutliers, 1, trainpreprocessed, type, nSetting, windowsize)
                testoutliers = np.apply_along_axis(calculateoutliers, 0, testpreprocessed, type, nSetting, windowsize)
                for histnorm in histnormalise:
                    for bucket in buckets:
                        print(datetime.now(), "histnorm: ", histnorm, "bucket: ", bucket)
                        Xtrain, minval, maxval = encode(trainoutliers, type, bucket, histnorm)
                        Xtest, _, _ = encode(testoutliers, type, bucket, histnorm, minval=minval, maxval=maxval)
                        # Add encode for test as well
                        for n_neighbors in k:
                            score, precision, recall = classify(Xtrain, Ytrain, Xtest, Ytest, n_neighbors)
                            nextRow =  pd.DataFrame([[type, sharpSetting, windowsize, nSetting, bucket, histnorm, n_neighbors, score, precision, recall]], columns=columns)
                            results = pd.concat([results, nextRow])
                        results.to_csv('out2.csv', index=False, header=True)


2022-09-13 10:24:30.553462 type:  scatter
2022-09-13 10:24:30.553686 sharpSetting:  direct
2022-09-13 10:25:15.169832 windowsize:  1
2022-09-13 10:25:15.169944 nSetting:  grid


error: OpenCV(4.6.0) :-1: error: (-5:Bad argument) in function 'filter2D'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'


In [46]:
# Debug cell

type = "scatter"
sharpSetting = "direct"
windowSize = 3
preprocessed = preprocess(integrals, objectinfo, type, sharpSetting, windowsize)
# preprocessed = preprocess(integrals, objectinfo, type, sharpSetting, windowsize)
nSetting = "euclidean"
neighbourhood = makeNeighbourhood(preprocessed, nSetting, windowsize) 
# neighbourhood = makeNeighbourhood(preprocessed, nSetting, windowsize)
histnorm = "none"
bucket = 30
Xtrain, Ytrain = encode(preprocessed, objectinfo, neighbourhood, windowsize, type, bucket, histnorm)
# Add encode for test as well
# for n_neighbors in k:
#     score, precision, recall = classify(Xtrain, Ytrain, Xtest, Ytest, n_neighbors)
#     nextRow =  pd.DataFrame([[type, sharpSetting, windowsize, nSetting, bucket, histnorm, n_neighbors, score, precision, recall]], columns=columns)
# results.to_csv('out-2.csv', index=False, header=True)


In [17]:
type = "scatter"
sharpSetting = "direct"
windowSize = 1
# preprocessed = preprocess(integrals, objectinfo, type, sharpSetting, windowsize)
testpreprocessed = preprocess(testintegrals, testobjectinfo, type, sharpSetting, windowsize)
nSetting = "euclidean"
# neighbourhood = makeNeighbourhood(preprocessed, nSetting, windowsize) 
testneighbourhood = makeNeighbourhood(testpreprocessed, nSetting, windowsize)
histnorm = "none"
bucket = 30
print()
# Xtrain, Ytrain = encode(preprocessed, objectinfo, neighbourhood, windowsize, type, bucket, histnorm)
Xtest, Ytest = encode(testpreprocessed, testobjectinfo, testneighbourhood, windowsize, type, bucket, histnorm)
# Add encode for test as well
# for n_neighbors in k:
#     score, precision, recall = classify(Xtrain, Ytrain, Xtest, Ytest, n_neighbors)
#     nextRow =  pd.DataFrame([[type, sharpSetting, windowsize, nSetting, bucket, histnorm, n_neighbors, score, precision, recall]], columns=columns)
# results.to_csv('out-2.csv', index=False, header=True)



0 14 0 1
(14, 122, 83) (14, 122, 83) (141764,) (141764,)
0 14 0 1
(14, 122, 83) (14, 122, 83) (141764,) (141764,)
0 14 0 1
(14, 122, 83) (14, 122, 83) (141764,) (141764,)
0 14 0 1
(14, 122, 83) (14, 122, 83) (141764,) (141764,)
0 14 0 1
(14, 122, 83) (14, 122, 83) (141764,) (141764,)
0 14 0 1
(14, 122, 83) (14, 122, 83) (141764,) (141764,)
0 14 0 1
(14, 122, 83) (14, 122, 83) (141764,) (141764,)
0 14 0 1
(14, 122, 83) (14, 122, 83) (141764,) (141764,)
0 14 0 1
(14, 122, 83) (14, 122, 83) (141764,) (141764,)
0 14 0 1
(14, 122, 83) (14, 122, 83) (141764,) (141764,)
0 14 0 1
(14, 122, 83) (14, 122, 83) (141764,) (141764,)
0 14 0 1
(14, 122, 83) (14, 122, 83) (141764,) (141764,)
0 14 0 1
(14, 122, 83) (14, 122, 83) (141764,) (141764,)
0 14 0 1
(14, 122, 83) (14, 122, 83) (141764,) (141764,)
0 14 0 1
(14, 122, 83) (14, 122, 83) (141764,) (141764,)
0 14 0 1
(14, 122, 83) (14, 122, 83) (141764,) (141764,)
0 14 0 1
(14, 122, 83) (14, 122, 83) (141764,) (141764,)
0 14 0 1
(14, 122, 83) (14, 12

ValueError: too many values to unpack (expected 2)